# Technical demand response potential meta analysis - method comparison (overview)

**Puporse and background**: This notebook serves for evaluating the methods of different publications for assessing technical demand response potentials for Germany. The data from the publications has been collected in order to carry out a meta analysis. The main use case of this particular notebook is to create bar charts for the methological elements evaluated. The method comparison analysis self has widely already been carried out using Excel.

## Method applied
In the following, a brief description of the method applied is given for the sake of replicability.

### Selection of publications
Criterions for the selection of publications were:
* Regional focus: Federal republic of Germany
* Temporal restricitions: Publications until 2005
* Contents: technical demand response potentials
* Sectoral coverage: At least one of households, trade, commerce and services or industry assessed; no publications focusing on single appliances
* Own analysis carried out (no other meta analysis included)
* Inclusion of at least one of the parameters of interest (usually at least some information on potential in MW)

### Methological elements evaluated
The following method elements have been evaluated and data has been collected and processed within an Excel workbook:
* overall methological approach applied and data sources used
* main assumptions made for potential determination
* data basis in detail
* method for further processing the potentials derived
* processes and appliances evaluated (per sector)
* parameters given to describe the technical demand response potential. These include:
    * capacity-related parameters: capacity available for load shifting or load shedding as well as minimum and maximum load level
    * time-reletad parameters: duration for activation, interference time, shifting time, regenaration time, time-dependent availability of potentials
    * cost-related parameters: specific investments, variable costs (for activation), fixed costs
    * These parameters have been separately collected in another Excel workbook (Potenziale_Lastmanagement.xlsx) and processed using another jupyter notebook (DR_potential_evaluation_VXX.ipynb).
* kind of potential evaluated in the study (besides the technical potential, such as theoretical, economic etc.)
* temporal horizon covered
* base year

### Filtering and data collection
* A total of 70 publications has been selected in the first place in a literature research based on the titles, keywords and abstracts using terms like "Demand Response", "Demand Side Management", "Demand Side Integration", "load management", "load shifting", "load shedding", "potential", "Germany" as well as their German equivalents to formulate search strings. 
* Of these publications, 30 have met the above stated criteria and were further evaluated. 
* Because sometimes, several publications belonged to the same research project, these were grouped together resulting in 24 publications resp. publication groups for which potential information has been evaluated.
* The data for the method comparison has been collected in an Excel Workbook (Methodenvergleich_Potenzialstudien.xlsx). The latest version of this Excel Workbook is evaluated with this notebook.

# Package imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Parameter settings
Set some parameters here in order to ensure code adaptability to future versions of potential collection (Excel workbook).

In [ ]:
path_folder_in = "./inputs/"
path_folder_out = "./out/"
path_folder_plots = "plots/"
path_folder_sources = "sources/"
filename = "Methodenvergleich_Potenzialstudien.xlsx"
filename_plot = "method_comparison"

# boolean parameters for controlling function behaviour
savefig = True
IEEE = False

# Read in data
Read in the relevant sheets of the Excel Workbook containing all data to be processed.

In [ ]:
# Read in data using pandas and parse it to individual DataFrames for each publication
xls = pd.ExcelFile(f"{path_folder_in}{filename}")

method = xls.parse("01_Methode_Daten_kodiert", skiprows=0, header=[1], index_col=0)
assumptions = xls.parse("01_Annahmen_Daten_kodiert", skiprows=0, header=[1], index_col=0)
data = xls.parse("01_Datenbasis_kodiert", skiprows=0, header=[1], index_col=0)
analysis = xls.parse("01_Folgeanalyse_kodiert", skiprows=0, header=[1], index_col=0)
citation = xls.parse("01_Zitationsanalyse_kodiert", skiprows=0, header=[1], index_col=0)
ind_processes = xls.parse("02_Ind_Prozesse_kodiert", header=[0], index_col=0)
ind_crosscutting = xls.parse("02_Ind_QST_kodiert", header=[0], index_col=0)
tcs_crosscutting = xls.parse("02_GHD_QST-Branchen_kodiert", header=[0], index_col=0)
hoho_appliances = xls.parse("02_HaHa_kodiert", header=[0], index_col=0)
flex_parameters = xls.parse("03_Flexparameter_kodiert", skiprows=0, header=[1], index_col=0)
availability = xls.parse("03_Zeitverfuegbarkeit_kodiert", skiprows=0, header=[1], index_col=0)
kind_potential = xls.parse("04_Potenzialbegriff_kodiert", skiprows=0, header=[1], index_col=0)
time_horizon = xls.parse("05_Betrachtungshorizont_kodiert", skiprows=0, header=[1], index_col=0)
base_year = xls.parse("05_Basisjahr_kodiert", skiprows=0, header=[1], index_col=0)

# read in overview containing all sources information
if IEEE:
    overview = xls.parse("Gesamtueberblick", skiprows=0, skipfooter=1, header=[1], usecols=[el for el in range(1,81)])
else:
    overview = xls.parse("Gesamtueberblick", skiprows=0, skipfooter=1, header=[1], usecols=[el for el in range(1,81)], index_col=0)

# Prepare sources information
* Transform the sources information into IEEE style.
* Concat the strings.
* Write the results to Excel.

In [ ]:
cols_to_use = [el for el in overview.columns if "Quellen" in el]
if IEEE:
    for el in cols_to_use:
        overview[el] = "[" + (overview.index + 1).astype(str) + "], S. " + overview[el].astype(str) + "; "
else:
    for el in cols_to_use:
        overview[el] = overview.index + ", S. " + overview[el].astype(str) + "; "
overview.loc["sum"] = overview[cols_to_use].sum()
overview.loc["sum", cols_to_use].to_excel(f"{path_folder_out}{path_folder_sources}sources_method_comparison.xlsx")

# Prepare data for further analysis
* Define a dict for iterating over all DataFrames (worksheets) which has a string as key and a list as value consisting of
    * the DataFrame itself,
    * the columns to be removed for creating bar plots as well as
    * the xaxis title for the individual subplot.
* Drop column Häufigkeit and calculate it using pd builtins

In [ ]:
method_dict = {
    "method": [method, ['Quellen Methodik', 'Bezugsjahr(e) der Datenbasis', 'Quellen Daten'], "Methode und Datenquellen"],
    "assumptions": [assumptions, ['Quellen zentrale Annahmen'], "zentrale Annahmen"],
    "data": [data, ['Quellen Daten'], "Datenquellen"],
    "analysis": [analysis, [], "Folgeverwertung der technischen Potenzialschätzung"],
    "citation": [citation, ['Fundstellen'], "Zitationshäufigkeiten"],
    "ind_processes": [ind_processes, [], "Industrielle Prozesse"],
    "ind_crosscutting": [ind_crosscutting, [], "Industrielle Querschnittstechnologien"],
    "tcs_crosscutting": [tcs_crosscutting, [], "Anwendungen und Branchen im GHD-Sektor"],
    "hoho_appliances": [hoho_appliances, [], "Anwendungen im Haushaltssektor"],
    "flex_parameters": [flex_parameters, ['Quellen Flexibilitätsparameter'], "ausgewertete Flexibilitätsparameter"],
    "availability": [availability, ['Quellen Zeitverfügbarkeit'], "Angaben zur Zeitverfügbarkeit"],
    "kind_potential": [kind_potential, [], "Verwendeter Potenzialbegriff"],
    "time_horizon": [time_horizon, [], "abgedeckter Zeithorizont"],
    "base_year": [base_year, [], "Basisjahr"]
}

In [ ]:
# Drop "Häufigkeit" columns and calculate sum as well as frequency using built-in functions
# Row Scholz et al. 2014 is dropped since it has been decided to group this publication together with the publications from Gils, esp. Gils 2015
for k in method_dict.keys():
    method_dict[k][0] = method_dict[k][0].drop(["Sch14", "Häufigkeit"]).fillna(0).rename_axis(None)
    method_dict[k][0].loc["Summe"] = method_dict[k][0].sum()
    method_dict[k][0].loc["Häufigkeit \n(inkl. teilweise Erfüllung)"] = method_dict[k][0].iloc[:-1].astype(bool).sum()

# Visualize the data
* Plot the data using a bar chart plot.
* Save the plot(s) output to a file.

In [ ]:
%matplotlib inline

In [ ]:
SMALL_SIZE = 13
MEDIUM_SIZE = 15
BIGGER_SIZE = 16

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)    # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
for i, (k, v) in enumerate(method_dict.items()):
    
    fig, ax = plt.subplots(figsize=(15, 8))
    
    to_plot = v[0].drop(method_dict[k][1], axis=1).loc[["Summe", "Häufigkeit \n(inkl. teilweise Erfüllung)"]].T
    _ = ax.set_ylim([0, 34])
    _ = ax.set_yticks(range(0, 34, 2))
    _ = ax.set_xlabel(method_dict[k][2], labelpad=10)
    _ = ax.set_ylabel('absolute Häufigkeit', labelpad=10)
    _ = to_plot.plot(kind="bar", ax=ax, color=["lightgray", "black"], edgecolor="black", hatch="/", zorder=10) 
    _ = ax.grid(True, linestyle='--', linewidth=0.5, zorder=-1)

    plt.tight_layout()    
    plt.savefig(f"{path_folder_out}{path_folder_plots}{filename_plot}_{k}.png", dpi=300, bbox_inches="tight")
#plt.show()

Create some separate plots (for the presentation only)

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(15,13))
    
to_plot_1 = method_dict["ind_processes"][0].drop(method_dict["ind_processes"][1], axis=1).loc[["Summe", "Häufigkeit \n(inkl. teilweise Erfüllung)"]].T
_ = to_plot_1.plot(kind="bar", ax=axs[0], color=["lightgray", "black"], hatch="/", zorder=10)
_ = axs[0].set_ylim([0, 34])
_ = axs[0].set_yticks(range(0, 34, 2))
_ = axs[0].set_ylabel('absolute Häufigkeit', labelpad=10)
_ = axs[0].grid(True, linestyle='--', linewidth=0.5, zorder=-1)

to_plot_2 = method_dict["ind_crosscutting"][0].drop(method_dict["ind_crosscutting"][1], axis=1).loc[["Summe", "Häufigkeit \n(inkl. teilweise Erfüllung)"]].T
_ = to_plot_2.plot(kind="bar", ax=axs[1], color=["grey", "black"], hatch="/", zorder=10)
_ = axs[1].set_ylim([0, 34])
_ = axs[1].set_yticks(range(0, 34, 2))
_ = axs[1].set_ylabel('absolute Häufigkeit', labelpad=10)
_ = axs[1].grid(True, linestyle='--', linewidth=0.5, zorder=-1)

plt.tight_layout()
plt.savefig(f"{path_folder_out}{path_folder_plots}{filename_plot}_industry.png", dpi=300, bbox_inches="tight")
#plt.show()

Print some information (for the presentation only)

In [ ]:
for k in method_dict.keys():
    print("------------------------------------------------------------------")
    print(k)
    print("------------------------------------------------------------------")
    display(method_dict[k][0].loc[["Summe", "Häufigkeit \n(inkl. teilweise Erfüllung)"]])